# Efficiency: threads
_COSC 208, Introduction to Computer Systems, 2023-11-17_

## Announcements
* Project 4 due Thurs, Dec 7

## Outline
* Warm-up
* Threads and memory
* Pthreads API

## Warm-up

* Q1: _What are all possible outputs produced by this program?_

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <unistd.h>
/* 4*/  void *proc1_main(void *arg) {
/* 5*/      int *x = (int *)arg;
/* 6*/      *x += 1;
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  void *proc2_main(void *arg) {
/*10*/      int *y = (int *)arg;
/*11*/      *y += 2;
/*12*/      return NULL;
/*13*/  }
/*14*/  int main() {
/*15*/      int *z = malloc(sizeof(int));
/*16*/      *z = 0;
/*17*/      int pid = fork();
/*18*/      if (pid == 0) {
/*19*/          proc1_main(z);
/*20*/      } else {
/*21*/          wait(NULL);
/*22*/          proc2_main(z);
/*23*/      }
/*24*/      printf("z is %d\n", *z);
/*25*/  }

z is 1
z is 2


<div style="height:7em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Threads and memory

* Threads are multiple execution contexts within the **same process**
    * Processes are multiple execution contexts within the **same machine**
* Because threads are within the same process, they share all of the process’s resources — memory, CPU time, file descriptors (i.e., open files), etc.
* Consequently, two threads can update the same variable

* Example

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *thread1_main(void *arg) {
/* 4*/      int *x = (int *)arg;
/* 5*/      *x += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *thread2_main(void *arg) {
/* 9*/      int *y = (int *)arg;
/*10*/      *y += 2;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *z = malloc(sizeof(int));
/*15*/      *z = 0;
/*16*/      // Create thread running thread1_main(z)
/*17*/      // Create thread running thread2_main(z)
/*18*/      // Wait for threads to finish
/*19*/      printf("z is %d\n", *z);
/*20*/  }

```
z is 3
```

* Two processes cannot update the same variable — memory is not shared; must use inter-process communication mechanism to share information

* Q2: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *increment(void *arg) {
/* 4*/      int *num = (int*)arg;
/* 5*/      *num += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *zero(void *arg) {
/* 9*/      int *num = (int*)arg;
/*10*/      *num = 0;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *i = malloc(sizeof(int));
/*15*/      *i = 5;
/*16*/      // Create thread running increment(i)
/*17*/      // Create thread running zero(i)
/*18*/      // Wait for threads to finish
/*19*/      printf("i=%d\n", *i);
/*20*/  }

```
i=0
```
OR
```
i=1
```

<p style="height:8em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Pthreads API

* Can create and wait for threads to finish, just like processes, but API is different
* Use the pthreads library—`#include <pthread.h>`
* `int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void*), void * arg)`
    * `thread`—a struct that stores metadata for the thread
    * `attr`—configuration settings for the thread
    * `start_routine`—the function to start executing when the thread starts
        * Pass a pointer to a function
    * `arg`—an argument passed to the aforementioned function
    * _How do we create a new process?_—`fork`
* `int pthread_join(pthread_t thread, void **value_ptr)`
    * `thread`—the same struct passed at thread creation; used to identify the thread we want to wait for
    * `value_ptr`—the location where the function return value should be stored
        * Notice it’s a pointer to a void pointer and the `start_routine` function specified in create returns a void pointer
    * _How do we wait for a process to finish?_ — `wait` or `waitpid`

* Q3: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'P';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      pthread_join(thread1, NULL);
/*15*/      *ch = 'Q';
/*16*/      pthread_create(&thread2, NULL, &printer, ch);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

    Postponed to next class

<div style="page-break-after:always;"></div>

## Extra practice

* Q4: _What output is produced by the following program? (Note: there is only one possible ordering.)_

In [2]:
/*1*/   #include <stdio.h>
/*2*/   #include <stdlib.h>
/*3*/   #include <sys/wait.h>
/*4*/   #include <unistd.h>
/*5*/   int main() {
/*6*/       printf("A\n");
/*7*/       int x = fork();
/*8*/       if (x == 0) {
/*9*/           int y = fork();
/*10*/          if (y == 0) {
/*11*/              printf("B\n");
/*12*/          }
/*13*/          else {
/*14*/              wait(NULL);
/*15*/              printf("C\n");
/*16*/          }
/*17*/      }
/*18*/      else {
/*19*/          wait(NULL);
/*20*/          printf("D\n");
/*21*/      }
/*22*/      printf("E\n");
/*23*/  }

A
B
E
C
E
D
E


<p style="height:8em;"></p>

* Q5: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *thread_main(void *arg) {
/* 4*/      char *id = (char *)arg;
/* 5*/      printf("I am thread %c\n", *id);
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  int main() {
/* 9*/      char *a = malloc(sizeof(char));
/*10*/      *a = 'A';
/*11*/      char *b = malloc(sizeof(char));
/*12*/      *b = 'B';
/*13*/      // Create thread running thread_main(a)
/*14*/      // Create thread running thread_main(b)
/*15*/      // Wait for threads to finish
/*16*/  }

```
I am thread A
I am thread B
```
OR
```
I am thread B
I am thread A
```